In [ ]:
pip install transformers

     |████████████████████████████████| 2.8 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.8 MB/s 
     |████████████████████████████████| 50 kB 9.0 MB/s 
     |████████████████████████████████| 636 kB 97.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn

import transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/MSR_data_0910/data_0910.zip

Archive:  drive/MyDrive/MSR_data_0910/data_0910.zip
   creating: data_0910/
  inflating: __MACOSX/._data_0910    
  inflating: data_0910/0906.csv      
  inflating: __MACOSX/data_0910/._0906.csv  
  inflating: data_0910/data_tensor_0910  
  inflating: __MACOSX/data_0910/._data_tensor_0910  
  inflating: data_0910/I_tensor0906  
  inflating: __MACOSX/data_0910/._I_tensor0906  


In [ ]:
!unzip drive/MyDrive/MSR_data_0910/data_0910.zip > /dev/null

replace __MACOSX/._data_0910? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
#url_df_read = pd.read_csv('drive/MyDrive/MSR_data_0831/df_csv_joined20180828.csv', index_col=[0])
#url_df_read

In [ ]:
#MSR_df_read = pd.read_csv('drive/MyDrive/MSR_data_0831/MSR_text_2.csv', index_col=[0])
#MSR_df_read

In [ ]:
import pandas as pd

df = pd.read_csv('drive/MyDrive/MSR_data_0910/0906.csv')
df

,Unnamed: 0,Text,Value,label
0,0,10/19/2020\nSLAVERY AND HUMAN TRAFFICKING\nINT...,"Whistleblower protection (direct employees), W...",1
1,1,Pension\nProtection\nFund\nPension\nProtection...,"Whistleblower protection (direct employees), I...",0
2,2,10/19/2020\nModern Slavery & Human\nTroffickin...,Whistleblower protection (direct employees),1
3,3,Greene King I MODERN SLAVERY STATEMENT\nMODERN...,Whistleblower protection (direct employees),1
4,4,Charles Wells Ltd\nModern Slavery and Human Tr...,Whistleblower protection (direct employees),1
...,...,...,...,...
1376,1376,Modern Slavery Act 2015 Statement\nOur Policy\...,No,0
1377,1377,Upto 60% off + 20% off ends\n\nSearch for\npro...,No,0
1378,1378,Slavery and Human Trafficking Transparency Sta...,Whistleblower protection (direct employees),1
1379,1379,Reward Gateway Slavery\nand Human Trafficking\...,"Hotline (direct employees), Whistleblower prot...",1


In [ ]:
#short_doc = []
#for i in range(len(df_csv_joined['Text'])):
  #if len(df_csv_joined['Text'][i]) < 1500:
    #short_doc.append(i)


In [ ]:
#df = df_csv_joined.drop(short_doc)
#df

In [ ]:
data_tensor = torch.load('drive/MyDrive/MSR_data_0910/data_tensor_0910')
#data_tensor_np = data_tensor.detach().numpy()
#data_tensor_np1 = np.delete(data_tensor_np, short_doc, axis=0)

#print(data_tensor_np1.shape)

#data_tensor_np2 = torch.tensor(data_tensor_np1)

print(data_tensor.size())

torch.Size([1381, 200, 768])


#### I Tensor

In [ ]:
I_tensor = torch.load('drive/MyDrive/MSR_data_0910/I_tensor0906')
#I_tensor_np = I_tensor.detach().numpy()
#I_tensor_np1 = np.delete(I_tensor_np, short_doc, axis=0)

#print(I_tensor_np1.shape)

#I_tensor1 = torch.tensor(I_tensor_np1)

print(I_tensor.size())


torch.Size([1381, 1, 200])


### Model training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
label = df['label'].tolist()
len(label)

1381

In [ ]:
label = df['label'].tolist()

train_x = data_tensor[:1000]
train_x = train_x.permute(0,2,1)
train_y = torch.tensor(label[:1000])
train_i = I_tensor[:1000]

val_x = data_tensor[1000:]
val_x = val_x.permute(0,2,1)
val_y = torch.tensor(label[1000:])
val_i = I_tensor[1000:]

test_x = data_tensor[1000:]
test_x = test_x.permute(0,2,1)
test_y = torch.tensor(label[1000:])
test_i = I_tensor[1000:]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 50

train_data = TensorDataset(train_x, train_y, train_i)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_x, val_y, val_i)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size, shuffle=False)

In [ ]:
class model(nn.Module):

    def __init__(self):
        
        super(model, self).__init__()
                        
        self.conv1 = nn.Conv1d(in_channels = 768, out_channels=1, kernel_size=1, stride=1, padding=0)
        
        self.conv2 = nn.Conv1d(in_channels = 768, out_channels=512, kernel_size=1, stride=1, padding=0)
        
        self.fc1 = nn.Linear(512, 100)
        
        self.fc2 = nn.Linear(100, 2)
                
        self.softmax1 = nn.Softmax(dim=-1)
        
        self.relu =  nn.ReLU()
        
        #self.dropout = nn.Dropout(0.01)
        

    #define the forward pass
    def forward(self, x, y, i):
        
        lbda = 5
                        
        alpha = self.conv1(x) #(btch_sz, 768, 200)-> (btch_sz, 1, 200)

        #print(alpha)
        
        x_2 = self.conv2(x)
        
        x_2 = self.relu(x_2)
                                                        
        alpha1 = alpha + lbda * i #(btch_sz, 1, 200) + (btch_sz,1, 200)*(btch_sz, 1, 200) 
                                
        alpha_prime = self.softmax1(alpha1) #(btch_sz, 1, 200)
                        
        alpha_prime_permute = alpha_prime.permute(0,2,1) #(btch_sz, 200, 1) 
                                        
        h = torch.matmul(x_2, alpha_prime_permute)  #(btch_sz, 768, 200)*(btch_sz, 200, 1) = (btch_sz, 768, 1)
                                
        h = h.view(h.shape[0], -1) #(btch_sz, 768)
                        
        output = self.relu(self.fc1(h)) #(btch_sz, 768, 512) 
        
        #output = self.dropout(output)
        
        output = self.fc2(output)
                                 
        return output

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
class_wts = compute_class_weight('balanced', classes= torch.unique(train_y).tolist(), y=train_y.tolist())

print(class_wts)

weights = torch.tensor(class_wts, dtype = torch.float)
weights = weights.to(device)


[1.01010101 0.99009901]


In [ ]:
from torch import optim

Model = model()

Model = Model.to(device)

cross_entropy = nn.CrossEntropyLoss(weight=weights)

from transformers import AdamW

optimizer = optim.AdamW(Model.parameters(), lr = 0.0005, weight_decay=0.01)

epochs = 375


In [ ]:
def train():
    Model.train()
    
    total_loss = 0

    train_correct = 0
    train_total = 0
    
    for step, batch in enumerate(train_dataloader):

        batch = [r.to(device) for r in batch]
 
        x, y, i = batch

        Model.zero_grad()        

        preds = Model(x, y, i)

        loss = cross_entropy(preds, y)

        total_loss = total_loss + loss.item()
     
        loss.backward()

        #torch.nn.utils.clip_grad_norm_(Model.parameters(), 1.0) 

        optimizer.step()  # update parameters

        _, predicted = torch.max(preds.data, 1)
        train_total += y.size(0)
        train_correct += (predicted == y).sum().item()

        preds=preds.detach().cpu().numpy()   # if model predictions are stored on GPU, push it to CPU
        
    avg_loss = total_loss / len(train_dataloader) # compute the training loss of the epoch
    train_acc = train_correct/train_total
  
    return avg_loss, train_acc #returns train loss and predictions


In [ ]:
def evaluate():
    
    print("\nEvaluating...")
    Model.eval()   # deactivate dropout layers
    
    total_loss = 0

    val_total = 0
    val_correct = 0

    for step,batch in enumerate(val_dataloader):   # iterate over batches
        
        batch = [t.to(device) for t in batch] # push the batch to gpu

        x, y, i = batch
        
        with torch.no_grad(): # deactivate autograd
            
            preds = Model(x, y, i)
            
            loss = cross_entropy(preds,y)

            total_loss = total_loss + loss.item()

            _, predicted = torch.max(preds.data, 1)
            val_total += y.size(0)
            val_correct += (predicted == y).sum().item()

            preds = preds.detach().cpu().numpy()
            
    avg_loss = total_loss / len(val_dataloader) 
    val_acc = val_correct/val_total

    return avg_loss, val_acc

In [ ]:
import numpy as np

#best_valid_loss = float('inf')  # set initial loss to infinite

best_valid_acc = 0

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

train_acces = []
val_acces = []

for epoch in range(epochs): #for each epoch
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() #train model
    valid_loss, val_acc = evaluate() #evaluate model
    
    #if valid_loss < best_valid_loss: #save the best model
        #best_valid_loss = valid_loss
        #torch.save(Model.state_dict(), 'saved_weights.pt')

    if val_acc > best_valid_acc: 
        best_valid_acc = val_acc
        torch.save(Model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))



 Epoch 1 / 375

Evaluating...
Train acc 0.597 Val acc 0.659
Train loss 0.677 Val loss 0.63

 Epoch 2 / 375

Evaluating...
Train acc 0.635 Val acc 0.659
Train loss 0.662 Val loss 0.622

 Epoch 3 / 375

Evaluating...
Train acc 0.651 Val acc 0.706
Train loss 0.648 Val loss 0.626

 Epoch 4 / 375

Evaluating...
Train acc 0.663 Val acc 0.693
Train loss 0.64 Val loss 0.606

 Epoch 5 / 375

Evaluating...
Train acc 0.653 Val acc 0.693
Train loss 0.636 Val loss 0.608

 Epoch 6 / 375

Evaluating...
Train acc 0.65 Val acc 0.672
Train loss 0.634 Val loss 0.612

 Epoch 7 / 375

Evaluating...
Train acc 0.668 Val acc 0.661
Train loss 0.638 Val loss 0.617

 Epoch 8 / 375

Evaluating...
Train acc 0.646 Val acc 0.696
Train loss 0.632 Val loss 0.605

 Epoch 9 / 375

Evaluating...
Train acc 0.651 Val acc 0.706
Train loss 0.626 Val loss 0.608

 Epoch 10 / 375

Evaluating...
Train acc 0.651 Val acc 0.693
Train loss 0.631 Val loss 0.607

 Epoch 11 / 375

Evaluating...
Train acc 0.661 Val acc 0.696
Train loss

In [ ]:
x = list(range(0,375))
plt.figure(figsize=(10,6))
plt.plot(x, train_acces)
plt.plot(x, val_acces)
plt.plot(x, train_losses)
plt.plot(x, valid_losses)

plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.title('Model train and validation loss')

In [ ]:
path = 'saved_weights.pt'
Model.load_state_dict(torch.load(path))

print('Train_report:')
with torch.no_grad():
  train_preds = Model(train_x.to(device), train_y.to(device), train_i.to(device))
  train_preds = train_preds.detach().cpu().numpy()

train_preds = np.argmax(train_preds, axis = 1)
print(classification_report(train_y, train_preds))


print('Val_report:')
with torch.no_grad():
  val_preds = Model(val_x.to(device), val_y.to(device), val_i.to(device))
  val_preds = val_preds.detach().cpu().numpy()

val_preds = np.argmax(val_preds, axis = 1)
print(classification_report(val_y, val_preds))

print('Test_report:')
with torch.no_grad():
  test_preds = Model(test_x.to(device), test_y.to(device), test_i.to(device))
  test_preds = test_preds.detach().cpu().numpy()

test_preds = np.argmax(test_preds, axis = 1)
print(classification_report(test_y, test_preds))

Train_report:
              precision    recall  f1-score   support

           0       0.72      0.76      0.74       495
           1       0.75      0.72      0.73       505

    accuracy                           0.74      1000
   macro avg       0.74      0.74      0.74      1000
weighted avg       0.74      0.74      0.74      1000

Val_report:
              precision    recall  f1-score   support

           0       0.74      0.56      0.63       151
           1       0.75      0.87      0.80       230

    accuracy                           0.75       381
   macro avg       0.74      0.71      0.72       381
weighted avg       0.74      0.75      0.74       381

Test_report:
              precision    recall  f1-score   support

           0       0.74      0.56      0.63       151
           1       0.75      0.87      0.80       230

    accuracy                           0.75       381
   macro avg       0.74      0.71      0.72       381
weighted avg       0.74      0.75  

In [ ]:
#import pickle

#with open('train_acc.data', 'wb') as file:
    #pickle.dump(train_acces, file)

In [ ]:
#with open('train_acc.data', 'rb') as file:
    #train_acc = pickle.load(file)